## 텍스트 유사도 


---
- 참고도서
  - 처음 배우는 딥러닝 챗봇 (조경래 지음 | 한빛미디어)
  - 밑바닥부터 시작하는 딥러닝 2 (사이토 고키 지음/개앞맵시 옮김 | 한빛미디어)
  - 자연어 처리 바이블 (임희석, 고려대학교 자연어처리연구실 지음 | 휴먼싸이언스)
  - 텐서플로 2와 머신러닝으로 시작하는 자연어 처리  (전창욱, 최태균, 조종현, 신성진 지음 | 위키북스)
- 참고사이트
  - https://koreapy.tistory.com/530
  - https://namu.wiki/w/노름(수학)
---

In [1]:
import torch 

### Norm 기반 유사도 

 Norm 
  - 선형대수에서의 Norm은 공간에 있는 모든 벡터의 총 길이
  - 거리의 일반화가 거리함수(distance function, 혹은 metric)라면 노름은 크기의 일반화를 가리킨다.
  - 실수의 크기(절댓값)를 $\lvert x \rvert$로 표현하듯, 벡터의 크기(노름)은 일반적으로 $\lVert{x}\rVert$라고 표현한다.
  - 저자에 따라서는 유클리드 공간의 노름을 $\lvert{x}\rvert$로 쓰기도 한다.

- Norm의 종류
  - L0 Norm
    - 실제 거리를 나타내는 Norm은 아님
    - 여러 개의 벡터 구성 요소 중에서 몇 개의 요소가 변화했는지, 올바르게 구성되어 있는지에 대한 확인을 위해 사용됨
    - 벡터에서 0이 아닌 요소의 총 수에 해당함
    - 예: 벡터 (0,0), (0,2)의 L0 Norm은 0이 아닌 요소가 하나뿐이므로 1
<br><br>
  - L1 Norm
    - 맨해튼 거리(Manhattan Distance/Norm) 또는 택시 거리(Taxicab Norm)라고 부르기도 함
    - 두 벡터의 각 차원별 값의 차이의절대값을 모두 합한 값이며, 일반적인 거리가 아닌 특이한 거리을 표시하는 특징을 가짐
    - 예: 좌표공간에 A(3,4)라는 점이 있다면 일반적인 (원점으로부터의)거리는
$\sqrt{{3^2}+{4^2}}=5$ 이지만 L1 Norm은 $3+4=7$임
    - 계산 방법: $\lVert{x}\rVert_1=\lvert{3}\rvert+\lvert{4}\rvert=7$
    - <img src='https://drive.google.com/uc?export=download&id=1GUXEIgE_5gWzWnhtQcaTJ9GJdeoeR2nh'>
$$
\text{d}_{\text{L1}}(w,v)=\sum_{i=1}^d{|w_i-v_i|},\text{ where }w,v\in\mathbb{R}^d.
$$
<br><br>
  - L2 Norm
    - 가장 널리 사용되는 표준 거리로 유클리드 공간(Euclidean Space)에 존재하기때문에 유클리디안 거리(Euclidean Distance)라고도 부름
    - 잘 알려진 피타고라스의 정리에 따른 거리 계산 방법
    - 한 지점에서 다른 지점으로 이동할 수 있는 최단 거리
    - 계산 방법: $\lVert{x}\rVert_2=\sqrt{{\lvert{3}\rvert}^2+{\lvert{4}\rvert}^2}=\sqrt{9+16}=\sqrt{25}=5$
    
    - <img src='https://drive.google.com/uc?export=download&id=1OdiWb0n8IiweZcLicjlNasjM7le7oNYZ'>
    - 벡터의 각 구성요소가 제곱 값으로 계산되므로, 특이값들이 일반적인 값보다 더 많은 가중치를 가지게 되어 결과를 왜곡하기 쉬움 → 일부 상황에서는 L1 Norm을 사용하게 되는 이유가 됨
    - 선형회귀의 선형성을 이탈하는 데이터 점(특이점)들에 대한 왜곡이 선형회귀식에서는 매우 큰 오류로 작용하게 되므로 그러한 경우에 L1 Norm을 사용하도록 설정하고 있지만 실무에서는 그냥 특이값은 제거해 버리는 편임
$$
\text{d}_{\text{L2}}(w,v)=\sqrt{\sum_{i=1}^d{(w_i-v_i)^2}},\text{ where }w,v\in\mathbb{R}^d.
$$
<br><br>
  - L-infinity Norm ($L_∞$ Norm)
    - 벡터공간에서 가장 큰 크기를 제공하는 Norm
    - 머신러닝, 딥러닝 분야에서는 그다지 취급하지 않음
$$
d_{\infty}(w,v)=\max(|w_1-v_1|,|w_2-v_2|,\cdots,|w_d-v_d|),\text{ where }w,v\in\mathbb{R}^d
$$

In [2]:
# L1 Distance 

def L1_Distance (x1 ,x2) :
    return ((x1 - x2).abs().sum())

In [3]:
# L2 Distance
def L2_Distance(x1, x2):
    return ((x1 - x2)**2).sum()**.5

In [4]:
# L-Infinity Distance

def infinity_distance(x1, x2):
    return ((x1 - x2).abs()).max()

### n-gram 유사도 

- n-gram
  - 주어진 문장에서 n개의 연속적인 단어 시퀀스(나열)를 의미
  - n-gram은 문장에서 n개의 단어를 토큰으로 사용함
  - 이웃한 단어의 출현 횟수를 통계적으로 표현해 텍스트의 유사도를 계산하는 방법
  - $\begin{aligned}Similarity=\frac{tf(A,B)}{tokens(A)}\end{aligned}$

- n-gram을 이용한 문장 간의 유사도 계산
    - 해당 문장을 n-gram으로 토큰 분리
    - 단어 문서 행렬(Term-Document Matrix, TDM) 생성
    - 두 문장 간 비교, 동일한 단어의 출현빈도를 확률로 계산
    - 계산 결과가 1.0에 가까울수록 B가 A과 유사하다고 볼 수 있음

In [5]:
from konlpy.tag import Komoran

In [13]:
# 어절 단위 n-gram ,문장을 이루는 마디

def word_ngram(bow, num_gram):
    text = tuple(bow)
    ngrams = [text[x:x + num_gram] for x in range(0, len(text))]
    return tuple(ngrams)

In [7]:
# 음절 n-gram 분석
def phoneme_ngram(bow, num_gram):
    sentence = ' '.join(bow)
    text = tuple(sentence)
    slen = len(text)
    ngrams = [text[x:x + num_gram] for x in range(0, slen)]
    return ngrams

In [8]:
# 유사도 계산
def similarity(doc1, doc2):
    cnt = 0
    for token in doc1:
        if token in doc2:
            cnt = cnt + 1

    return cnt/len(doc1)

In [9]:
# 문장 정의 

sentence1 = "한미 정상은 이번 회담에서 반도체·배터리·인공지능(AI) 등 핵심·신흥기술 협력과 안전하고 지속 가능하며 회복력 있는 글로벌 공급망을 위해서도 공조하기로 했다."
sentence2 = '윤 대통령과 바이든 대통령은 세계 최대 반도체 생산시설인 삼성 평택캠퍼스를 함께 시찰하며 반도체 동맹 행보에 나섰다.'

In [14]:
# 형태소분석기를 이용해 단어 묶음 리스트 생성

komoran = Komoran()

bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)

doc1 = word_ngram(bow1 , 2)
doc2 = word_ngram(bow2, 2)

print(doc1)
print(doc2)

(('미', '정상은'), ('정상은', '이번'), ('이번', '회담'), ('회담', '반도체'), ('반도체', '배터리'), ('배터리', '인공지능'), ('인공지능', '등'), ('등', '핵심'), ('핵심', '신흥'), ('신흥', '기술'), ('기술', '협력'), ('협력', '안전'), ('안전', '지속'), ('지속', '회복'), ('회복', '력'), ('력', '글로벌'), ('글로벌', '공급'), ('공급', '망'), ('망', '공조'), ('공조',))
(('윤', '대통령'), ('대통령', '바이든'), ('바이든', '대통령'), ('대통령', '세계'), ('세계', '최대'), ('최대', '반도체'), ('반도체', '생산'), ('생산', '시설'), ('시설', '삼성'), ('삼성', '평택'), ('평택', '캠퍼스'), ('캠퍼스', '시찰'), ('시찰', '반도체'), ('반도체', '동맹'), ('동맹', '행보'), ('행보',))


In [16]:
r1 = similarity(doc1, doc2)
print(r1)

0.0


### 코사인 유사도 

- 두 벡터 사이의 요소별 곱을 사용하여 거리를 계산하는 방법으로 벡터의 내적과 같음
- 두 벡터 사이의 방향과 크기를 모두 고려함
- 코사인 유사도의 결과가
  - 1에 가까울수록 방향은 일치
  - 0에 가까울수록 직교
  - -1에 가까울수록 반대 방향
- 연산에 대한 부하가 큼
- 희소벡터일 경우 큰 문제가 발생함
  - 윗변이 벡터의 곱으로 표현되므로 0이 들어간 차원이 많으면 해당 차원이 직교하면서 곱의 값이 0이 됨
  - 따라서 정확한 유사도 또는 거리를 계산하지 못함

$$
\begin{aligned}
\text{sim}_{\text{cos}}(w,v)&=\overbrace{\frac{w\cdot v}{|w||v|}}^{\text{dot product}}
=\overbrace{\frac{w}{|w|}}^{\text{unit vector}}\cdot\frac{v}{|v|} \\
&=\frac{\sum_{i=1}^{d}{w_iv_i}}{\sqrt{\sum_{i=1}^d{w_i^2}}\sqrt{\sum_{i=1}^d{v_i^2}}} \\
\text{where }&w,v\in\mathbb{R}^d
\end{aligned}
$$

- 단어나 문장을 벡터로 표현할 수 있다면 벡터간 거리나 각도를 이용해서 단어, 문장 사이의 유사성을 계산할 수 있음
- 코사인 유사도는 벡터의 크기가 중요하지 않을 때 그 거리를 측정하기 위하여 많이 사용됨
- 단어들의 출현 빈도를 통해 유사도 계산을 한다면
  - 동일한 단어가 많이 포함되어 있을수록 벡터의 크기가 커짐
  - 그러나 코사인 유사도는 벡터의 크기와 상관없이 결과가 안정적이므로 좋은 결과를 기대할 수 있음

In [17]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [18]:
# 코사인 유사도 계산
def cos_sim(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

In [19]:
# 코사인 유사도 계산 (다른 버전)
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / ((x1**2).sum()**.5 * (x2**2).sum()**.5)

단어 문서 행렬(Term-Document Matrix, TDM)이란 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것

In [20]:
# TDM 만들기
def make_term_doc_mat(sentence_bow, word_dics):
    freq_mat = {}

    for word in word_dics:
        freq_mat[word] = 0

    for word in word_dics:
        if word in sentence_bow:
            freq_mat[word] += 1

    return freq_mat

In [21]:
# 단어 벡터 만들기
def make_vector(tdm):
    vec = []
    for key in tdm:
        vec.append(tdm[key])
    return vec

In [22]:
# 단어 묶음 리스트를 하나로 합침
bow = bow1 + bow2 

In [23]:
# 단어 묶음에서 중복제거해 단어 사전 구축
word_dics = []
for token in bow:
    if token not in word_dics:
        word_dics.append(token)

In [24]:
# 문장 별 단어 문서 행렬 계산
freq_list1 = make_term_doc_mat(bow1, word_dics)
freq_list2 = make_term_doc_mat(bow2, word_dics)

print(freq_list1)
print(freq_list2)

{'미': 1, '정상은': 1, '이번': 1, '회담': 1, '반도체': 1, '배터리': 1, '인공지능': 1, '등': 1, '핵심': 1, '신흥': 1, '기술': 1, '협력': 1, '안전': 1, '지속': 1, '회복': 1, '력': 1, '글로벌': 1, '공급': 1, '망': 1, '공조': 1, '윤': 0, '대통령': 0, '바이든': 0, '세계': 0, '최대': 0, '생산': 0, '시설': 0, '삼성': 0, '평택': 0, '캠퍼스': 0, '시찰': 0, '동맹': 0, '행보': 0}
{'미': 0, '정상은': 0, '이번': 0, '회담': 0, '반도체': 1, '배터리': 0, '인공지능': 0, '등': 0, '핵심': 0, '신흥': 0, '기술': 0, '협력': 0, '안전': 0, '지속': 0, '회복': 0, '력': 0, '글로벌': 0, '공급': 0, '망': 0, '공조': 0, '윤': 1, '대통령': 1, '바이든': 1, '세계': 1, '최대': 1, '생산': 1, '시설': 1, '삼성': 1, '평택': 1, '캠퍼스': 1, '시찰': 1, '동맹': 1, '행보': 1}


In [25]:
# 코사인 유사도 계산
doc1 = np.array(make_vector(freq_list1))
doc2 = np.array(make_vector(freq_list2))

r1 = cos_sim(doc1, doc2)
print(r1)

0.05976143046671968


In [26]:
r1 = get_cosine_similarity(doc1, doc2)

print(r1)

0.05976143046671968


In [27]:
# 유사 단어 랭킹 표시 

def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [28]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    '''동시발생 행렬 생성

    :param corpus: 말뭉치(단어 ID 목록)
    :param vocab_size: 어휘 수
    :param window_size: 윈도우 크기(윈도우 크기가 1이면 타깃 단어 좌우 한 단어씩이 맥락에 포함)
    :return: 동시발생 행렬
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

In [29]:
# 코사인 유사도 계산 (또 다른 버전)
def cos_similarity(x, y):
  nx = x / np.sqrt(np.sum(x ** 2))
  ny = y / np.sqrt(np.sum(y ** 2))
  return np.dot(nx, ny)

# 파라미터에 제로벡터가 들어오면 "Divided by Zero"오류 발생.
# 매우 작은 수 eps를 분모에 추가하여 회피
def new_cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

In [30]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']]  # "you"의 단어 벡터
c1 = C[word_to_id['i']]    # "i"의 단어 벡터
print(cos_similarity(c0, c1))

0.7071067811865475


In [31]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    '''유사 단어 검색

    :param query: 쿼리(텍스트)
    :param word_to_id: 단어에서 단어 ID로 변환하는 딕셔너리
    :param id_to_word: 단어 ID에서 단어로 변환하는 딕셔너리
    :param word_matrix: 단어 벡터를 정리한 행렬. 각 행에 해당 단어 벡터가 저장되어 있다고 가정한다.
    :param top: 상위 몇 개까지 출력할 지 지정
    '''
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 코사인 유사도 계산
    vocab_size = len(id_to_word)

    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [32]:
most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067811865475
 i: 0.7071067811865475
 hello: 0.7071067811865475
 say: 0.0
 and: 0.0
